<a href="https://colab.research.google.com/github/bominwang/Bayesian-statistics-method/blob/bayesian-model-averaging-method/bayesian_model_averaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [161]:
import numpy as np

In [160]:
class model_posterior(object):
  
  def __init__(self, experiment_data, model_dataset, model_prior):
    # experiment_data 和 model_data 都为numpy数组，每行表示一个样本
    # model_dataset为3维张量，model_dataset[i, :, :] = model_data
    # model_prior为1维张量 model_prior[i,:] = prior
    # warning：会出现后验概率和不为1的情况，请检查代码
    
    self.experiment_data = experiment_data
    self.model_dataset = model_dataset
    self.model_prior = model_prior
    
  
  def error_covariance(self, experiment_data, model_data):
    
    data_volume = experiment_data.shape[0]
    covariance_matrix = np.divide(np.dot((experiment_data - model_data), (experiment_data - model_data).T), data_volume)
    
    return covariance_matrix
  
  def model_likelihood(self, experiment_data, model_data):
    
    data_size = experiment_data.shape[1]
    data_volume = experiment_data.shape[0]
    covariance_matrix = self.error_covariance(experiment_data, model_data)
    likelihood = np.zeros([data_volume, 1])
    
    for i in range(data_volume):
      error = experiment_data[i, :] - model_data[i, :]
      likelihood[i,0] = np.power(2 * np.pi, (-(0.5 * data_size))) * np.power(np.linalg.det(covariance_matrix), -1) * np.exp(np.dot((np.dot(error.T, np.linalg.inv(covariance_matrix))), error))
    
    likelihood = np.sum(likelihood)
    
    return likelihood
  
  def model_posterior(self, experiment_data, model_dataset, model_prior):
    
    model_volume = model_prior.shape[0]
    posterior = np.zeros([model_volume, 1])
    likelihood = np.zeros([model_volume, 1])
    for i in range(model_volume):
      model_data = model_dataset[i, :, :]
      likelihood[i, :] = self.model_likelihood(experiment_data, model_data)
    
    normlization_factor = np.dot(model_prior.T, likelihood)
    
    for j in range(model_volume):
      posterior[i, :] = (likelihood[i, :] * model_prior[i, :]) / normlization_factor
    
    return posterior